## SPAM CLASSIFIER

This exercise is mostly about applying text preprocessing than actual spam classfication. We will be using Multinomial Naive Bayes for Classification. ( mostly cause of less data)

Why Stemming? The tradeoff is not worth it. NB is not

In [2]:
import pandas as pd

df = pd.read_csv("data/SMSSpamData/SMSSpamCollection", sep = '\t', names = ['Label', 'Message'])
df.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [35]:
import re
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
processed_messages = []

for i in range(len(df)):
    processed_message = re.sub('[^a-zA-z]', ' ', df['Message'][i])
    processed_message = processed_message.lower()
    processed_message = processed_message.split()
    
    processed_message = [stemmer.stem(word) for word in processed_message if not word in stopwords.words('english')]
    processed_message = ' '.join(processed_message)
    
    processed_messages.append(processed_message)

df['Processed Message'] = processed_messages
df.head()

,Label,Message,Processed Message
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entri wkli comp win fa cup final tkt st m...
3,ham,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think goe usf live around though


In [48]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(processed_messages).toarray()

y = pd.get_dummies(df['Label'])
y.drop(columns=['ham'], inplace = True)
y = y['spam'].values

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [51]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [55]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       959
           1       0.91      0.95      0.93       156

    accuracy                           0.98      1115
   macro avg       0.95      0.97      0.96      1115
weighted avg       0.98      0.98      0.98      1115

0.9802690582959641
